In [2]:
import pandas as pd



In [92]:
people = pd.read_csv('raw/person.csv', dtype={'Id': str}, parse_dates=[17, 18])

# people[people.Tom > '2019-01-01'][(people.Uppdragsroll == 'Ledamot')].drop_duplicates(subset='Id')
# people['Uppdragsroll'].unique()
uppdragstyp_values = people.Uppdragstyp.unique()
filters = (people.Uppdragstyp.isin(uppdragstyp_values[:-2]) 
           & (people.Tom >= '2020-02-01') 
           & (people.Status.str.contains('minister') == False)
           & (people.Status.str.contains('Tidigare') == False)
           & (~people.Status.isin(['Statsråd', 'Tjänstgörande statsrådsersättare', ]))
          )

people[filters].groupby('Status').count()

,Förnamn,Efternamn,Iort,Parti,Id,Kön,Född,Valkrets,Webbadress,Epostadress,Telefonnummer,Titel,Uppdragstyp,Uppdragsorgan,Uppdragsroll,Uppdragsrollstatus,From,Tom
Status,,,,,,,,,,,,,,,,,,
Andre vice talman (tjänstgörande riksdagsledamot),5,5,0,5,5,5,5,5,0,5,5,5,5,5,5,1,5,5
Chefsjustitieombudsman i Riksdagens ombudsmän,1,1,0,0,0,1,1,0,0,0,0,0,1,1,1,0,1,1
Ersättare för ordföranden i Riksdagens arvodesnämnd,2,2,0,0,0,2,2,0,0,0,0,0,2,2,2,0,2,2
Ersättare för ordföranden i Riksdagens överklagandenämnd,2,2,0,0,0,2,2,0,0,0,0,0,2,2,2,0,2,2
Ersättare i Riksdagens ansvarsnämnd,1,1,0,0,0,1,1,0,0,0,0,0,1,1,1,0,1,1
Förste vice talman (tjänstgörande riksdagsledamot),4,4,0,4,4,4,4,4,0,4,4,4,4,4,4,1,4,4
Justitieombudsman i Riksdagens ombudsmän,3,3,0,0,0,3,3,0,0,0,0,0,3,3,3,0,3,3
Ledamot i Nämnden för lön till riksdagens ombudsmän och riksrevisorerna,2,2,0,0,0,2,2,0,0,0,0,0,2,2,2,0,2,2
Ledamot i Nämnden för prövning av statsråds och statssekreterares övergångsrestriktioner,2,2,0,0,0,2,2,0,0,0,0,0,2,2,2,0,2,2


In [4]:
import os
import os.path as path
import json

def load_json(f):
    try:
        with open(f, 'r') as fd:
            return json.load(fd)
    except json.JSONDecodeError as e:
        print('Failed to read', f, "error:", e)

betankande_files = os.listdir('raw/bet-2018-2021.json/')
betankande_raw = {path.splitext(f)[0]: load_json(path.join('raw/bet-2018-2021.json/', f)) for f in betankande_files}

Failed to read raw/bet-2018-2021.json/h601ku7.json error: Expecting property name enclosed in double quotes: line 1 column 117544 (char 117543)
Failed to read raw/bet-2018-2021.json/h601uu10.json error: Expecting property name enclosed in double quotes: line 1 column 739291 (char 739290)
Failed to read raw/bet-2018-2021.json/h601ku14.json error: Expecting property name enclosed in double quotes: line 1 column 151742 (char 151741)


In [7]:
import funcy as fp

def to_list(x):
    return x if isinstance(x, list) else [] if x is None else [x]
        
def get_betankande_values(bet):
    dok = fp.get_in(bet, ['dokumentstatus', 'dokument'])
    dok_data = fp.project(dok, ['dok_id', 'publicerad', 'titel', 'subtitel', 'status'])
    dok_ut = to_list(fp.get_in(bet, ['dokumentstatus', "dokutskottsforslag", 'utskottsforslag']))
    dok_ut_data = fp.walk(lambda ut: fp.project(ut, ['punkt', 'rubrik', 'votering_id', 'beslutstyp', 'vinnare']), dok_ut)
    dok_bilaga = to_list(fp.get_in(bet, ['dokumentstatus', 'dokbilaga', 'bilaga']))
    attachments =  [{'url': fp.get_in(b, ['fil_url']), 'titel': fp.get_in(b, ['titel'])} for b in dok_bilaga
                    if b['filtyp'] == 'pdf']
    references = [{'type': r['ref_dok_typ'], 'dok_id': r['ref_dok_id']} 
                  for r
                  in to_list(fp.get_in(bet, ['dokumentstatus', 'dokreferens', 'referens']))
                  if r['ref_dok_typ'] in ('mot', 'prop')
                 ]
    return fp.merge(dok_data, 
                    { 'points': dok_ut_data,
                     'attachments': attachments,
                     'references': references
                    })

betankande = fp.walk(lambda b: (b[1]['dok_id'], b[1]), 
                                  fp.walk_values(get_betankande_values, fp.compact(betankande_raw)))

with open('betankande.json', 'w') as fd:
    json.dump(betankande, fp=fd, ensure_ascii=False)
    
# betankande_text = {fp.get_in(d, ['dokumentstatus', 'dokument', 'dok_id']):
#                    {'dok_id': fp.get_in(d, ['dokumentstatus', 'dokument', 'dok_id']),
#                    'html': fp.get_in(d, ['dokumentstatus', 'dokument', 'html'])}
#                   for d in betankande_raw.values()
#                    if d is not None
#                   }

# with open('betankande_text.json', 'w') as fp:
#     json.dump(betankande_text, fp=fp, ensure_ascii=False)

In [18]:
mot_files = os.listdir('raw/mot-2018-2021.json/')
mot_raw = [load_json(path.join('raw/mot-2018-2021.json/', f)) for f in mot_files]

get = lambda path, col: fp.get_in(col, path)

def get_mot_values(mot):
    data = get(['dokumentstatus'], mot)
    dok = {'dok_id': get(['dokument', 'dok_id'], data),
           'organ': get(['dokument', 'organ'], data),
           'publicerad': get(['dokument', 'publicerad'], data),
           'subtyp': get(['dokument', 'subtyp'], data)
          }
    intressents = to_list(get(['dokintressent', 'intressent'], data))
    intressents_data = [{'intressent_id': get(['intressent_id'], d),
                         'namn': get(['namn'], d),
                         'partibet': get(['partibet'], d)
                        } 
                        for d in intressents
                       ]
    dok_bilaga = to_list(fp.get_in(data, ['dokbilaga', 'bilaga']))
    attachments =  [{'url': fp.get_in(b, ['fil_url']), 
                     'titel': fp.get_in(b, ['titel'])
                    } for b in dok_bilaga
                      if b['filtyp'] == 'pdf'
                   ]
    dok_referens = to_list(get(['dokreferens', 'referens'], data))
    references = [{'type': r['ref_dok_typ']
                   , 'dok_id': r['ref_dok_id']
                  } 
                  for r in dok_referens
                  if r['ref_dok_typ'] in ('bet')
                 ]
    return fp.merge(dok, {'attachments': attachments, 'references': references, 'intressent': intressents_data})
    

motions = {data['dok_id']: data for data in map(get_mot_values, mot_raw)}

with open('motions.json', 'w') as fd:
    json.dump(fp=fd, obj=motions)


In [25]:
prop_files = os.listdir('raw/prop-2018-2021.json/')
prop_raw = [load_json(path.join('raw/prop-2018-2021.json/', f)) for f in prop_files]


def get_prop_values(prop):
    data = get(['dokumentstatus'], prop)
    dok = {'dok_id': get(['dokument', 'dok_id'], data),
           'organ': get(['dokument', 'organ'], data),
           'publicerad': get(['dokument', 'publicerad'], data),
           'mottagare': get(['dokument', 'mottagare'], data),
           'subtyp': get(['dokument', 'subtyp'], data),
          }
    intressents = to_list(get(['dokintressent', 'intressent'], data))
    intressents_data = [{'intressent_id': get(['intressent_id'], d),
                         'namn': get(['namn'], d),
                         'partibet': get(['partibet'], d)
                        } 
                        for d in intressents
                       ]
    dok_bilaga = to_list(fp.get_in(data, ['dokbilaga', 'bilaga']))
    attachments =  [{'url': fp.get_in(b, ['fil_url']), 
                     'titel': fp.get_in(b, ['titel'])
                    } for b in dok_bilaga
                      if b['filtyp'] == 'pdf'
                   ]
    dok_referens = to_list(get(['dokreferens', 'referens'], data))
    references = [{'type': r['ref_dok_typ']
                   , 'dok_id': r['ref_dok_id']
                  } 
                  for r in dok_referens
                  if r['ref_dok_typ'] in ('bet', 'mot')
                 ]
    return fp.merge(dok, {'attachments': attachments, 'references': references, 'intressent': intressents_data})

propositions = {data['dok_id']: data for data in map(get_prop_values, prop_raw)}

with open('propositions.json', 'w') as fd:
    json.dump(fp=fd, obj=propositions)

In [53]:
votering_dirs = ['raw/votering-201819.json/', 'raw/votering-201920.json/']
votering_files = [os.path.join(d, f) for d in votering_dirs for f in os.listdir(d)]

def get_betankande_and_punkt(fname):
    base = os.path.basename(fname).split('-')
    return base[0], base[1]

votering_raw = {get_betankande_and_punkt(f): load_json(f) for f in votering_files}

def get_votering_values(key, vot):
    votes_data = get(['dokvotering', 'votering'], vot)
    votes = [{'intressent_id': v['intressent_id'],
              'vote': v['rost']
             } 
             for v in votes_data
            ]
    return {
        'votering_id': get([0, 'votering_id'], votes_data).lower(),
        'votes': votes,
        'bet': key[0],
        'punkt': key[1],
        'type': get([0, 'votering'], votes_data),
        'regarding': get([0, 'avser'], votes_data),
        'date': get([0, 'datum'], votes_data)
    }

votes_summary = [get_votering_values(k, v) for (k, v) in votering_raw.items()]

In [57]:
votering_id_indexed = {v['votering_id']: v for v in votes_summary}
votering_by_bet = fp.group_by(lambda v: v['bet'], votes_summary)

flattened = [fp.merge(
                fp.project(v, ['votering_id', 'bet', 'punkt', 'type', 'regarding', 'date']),
                vote
                )
             for v in votes_summary
             for vote in v['votes']
            ]

votes_by_voter = fp.group_by(lambda v: v['intressent_id'], flattened)

len(votes_by_voter)

393

# 